# Detecting Irony Using Google Universal Sentence Encoder
In this notebook, I will attempt to detect irony using Google's Universal Sentence Encoder 

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import keras.backend as K
from keras.layers import *
from keras.models import *
from keras.utils import *
from keras.regularizers import *
import numpy as np
import _pickle as pickle
import uuid

W0427 18:26:25.637270  9944 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [2]:
goodreads_irony_f = open("goodreads_irony_edited.pickle", mode="rb")
goodreads_knowledge_f = open("goodreads_knowledge_keep.pickle", mode="rb")
goodreads_metaphor_f = open("goodreads_metaphor_keep.pickle", mode="rb")

goodreads_irony = pickle.load(goodreads_irony_f)
goodreads_knowledge = pickle.load(goodreads_knowledge_f)
goodreads_metaphor = pickle.load(goodreads_metaphor_f)

goodreads_irony_f.close()
goodreads_knowledge_f.close()
goodreads_metaphor_f.close()

ironic = goodreads_irony
ironic_labels = np.ones(len(ironic))
non_ironic = goodreads_knowledge + goodreads_metaphor
non_ironic_labels = np.zeros(len(non_ironic))
full_set = ironic + non_ironic
full_labels = np.concatenate([ironic_labels, non_ironic_labels])

ironic_size = len(ironic)
non_ironic_size = len(non_ironic)
full_set_size = len(full_set)

print("Ironic examples: " + str(ironic_size))
print("Non ironic examples: " + str(non_ironic_size))
print("Full set (for validation): " + str(full_set_size))

Ironic examples: 320
Non ironic examples: 807
Full set (for validation): 1127


In [3]:
full_set_np = np.asarray(full_set)

train_features = np.concatenate([full_set_np[0:240], full_set_np[320:925]])
train_labels = np.concatenate([np.ones(240), np.zeros(605)])

test_features = np.concatenate([full_set_np[240:320], full_set_np[925:1127]])
test_labels = np.concatenate([np.ones(80), np.zeros(202)])

print(len(train_features))
print(len(train_labels))
print(len(test_features))
print(len(test_labels))

845
845
282
282


In [4]:
train_features = np.reshape(train_features, (train_features.shape[0],1))
test_features = np.reshape(test_features, (test_features.shape[0],1))
x = np.reshape(full_set_np, (full_set_np.shape[0],1))
y = to_categorical(full_labels)

In [5]:
embed = hub.Module('./embeddings/GUSE')
test_messages = ["The quick brown fox jumped over the lazy dog"]

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    message_embeddings = session.run(embed(test_messages))
message_embeddings

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0427 18:26:34.499387  9944 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


array([[ 1.64562017e-02,  6.23100176e-02,  3.46776913e-03,
         4.70204279e-03,  3.23413201e-02,  9.14165005e-03,
        -3.01914979e-02, -3.84491310e-02, -3.57649252e-02,
        -4.20018146e-03, -1.91205703e-02,  2.40280163e-02,
         5.23959063e-02,  2.04604957e-02,  3.24646309e-02,
         8.14204291e-02, -3.86404321e-02,  1.64812542e-02,
         1.34157818e-02, -3.47102135e-02,  7.75472447e-02,
        -7.51131922e-02,  1.21427458e-02,  4.46278341e-02,
         7.48834983e-02, -1.19225457e-02, -2.63228826e-02,
         1.70647651e-02,  7.75867552e-02,  4.95012477e-02,
         3.56119871e-02,  4.83054519e-02, -1.33931926e-02,
        -2.03491952e-02,  1.61516182e-02, -6.49256110e-02,
        -3.63756418e-02, -3.20054255e-02,  4.01207134e-02,
         5.42915612e-02, -4.16404940e-02, -5.53556010e-02,
        -6.92512095e-02,  4.85552102e-03,  1.49620827e-02,
         2.48402003e-02,  1.57571007e-02,  4.47167791e-02,
        -7.07037523e-02, -7.17832223e-02,  5.00952639e-0

In [6]:
def GUSE(param):
    return embed(tf.squeeze(tf.cast(param, tf.string)), signature="default", as_dict=True)["default"]

In [7]:
input_layer = Input(shape=(1,), dtype=tf.string)
guse = Lambda(GUSE, output_shape=(512,))(input_layer)
dense1 = Dense(256, activation="relu", kernel_regularizer=l2(0.005))(guse)
dropout1 = Dropout(0.6)(dense1)
dense2 = Dense(128, activation="relu", kernel_regularizer=l2(0.005))(dropout1)
dropout2 = Dropout(0.6)(dense2)
dense3 = Dense(128, activation="relu", kernel_regularizer=l2(0.005))(dropout2)
dropout3 = Dropout(0.6)(dense3)
dense4 = Dense(64, activation="relu", kernel_regularizer=l2(0.005))(dropout3)
dropout4 = Dropout(0.6)(dense4)
output = Dense(2, activation="softmax")(dropout4)
model = Model(inputs=[input_layer], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0427 18:26:36.872856  9944 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
__________

In [8]:
session = tf.Session()
K.set_session(session)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
history = model.fit(x, y, batch_size=64, epochs=30, validation_split=0.1, shuffle=True)
nonce = str(int(uuid.uuid4()))
model.save_weights('./models/guse_model' + nonce + '.h5')

Train on 1014 samples, validate on 113 samples
Epoch 1/30
1014/1014 [==============================] - 4s 4ms/step - loss: 3.8677 - acc: 0.6154 - val_loss: 3.2170 - val_acc: 1.0000
Epoch 2/30
1014/1014 [==============================] - 1s 1ms/step - loss: 2.9681 - acc: 0.6815 - val_loss: 2.3396 - val_acc: 1.0000
Epoch 3/30
1014/1014 [==============================] - 1s 1ms/step - loss: 2.2603 - acc: 0.6844 - val_loss: 1.7390 - val_acc: 1.0000
Epoch 4/30
1014/1014 [==============================] - 1s 1ms/step - loss: 1.7282 - acc: 0.6844 - val_loss: 1.2811 - val_acc: 1.0000
Epoch 5/30
1014/1014 [==============================] - 1s 999us/step - loss: 1.3333 - acc: 0.6854 - val_loss: 0.9524 - val_acc: 1.0000
Epoch 6/30
1014/1014 [==============================] - 1s 1ms/step - loss: 1.0854 - acc: 0.8393 - val_loss: 0.7551 - val_acc: 0.9558
Epoch 7/30
1014/1014 [==============================] - 1s 1ms/step - loss: 0.8832 - acc: 0.8876 - val_loss: 0.6395 - val_acc: 0.9558
Epoch 8/30
10

In [9]:
twitter_f = open("twitter_irony_all.pickle", mode="rb")
twitter_data = pickle.load(twitter_f)
a = np.zeros((156,1))
b = np.ones((156,1))
twitter_y = np.concatenate((a,b), axis=1)
twitter_x = np.asarray(twitter_data)
twitter_x = np.reshape(twitter_x, (twitter_x.shape[0],1))

In [10]:
scores = model.evaluate(twitter_x, twitter_y)

156/156 [==============================] - 0s 889us/step


In [11]:
scores[1]

0.7948717979284433